# Download & Import libraries

In [1]:
!pip install ultralytics
# !pip install -q kaggle
!gdown 1ez1Wf_62RlE4VfkuZm9JoUqWDrCrpHNS # Train Data 64 classes
!gdown 1w9NK059xnNzgpQTLpyLJwKCSV6ucbKfL # Test Data 1
!gdown 16wXoUU8MTQbgnYVgsa14vy00nTxwpcKs # Test Data 2
# !yolo predict model=yolov8n.pt source='https://ultralytics.com/images/bus.jpg' # test yolov8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 9.0 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=1ez1Wf_62RlE4VfkuZm9JoUqWDrCrpHNS
To: /content/LogoDet-64.zip
100% 49.3M/49.3M [00:01<00:00, 31.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w9NK059xnNzgpQTLpyLJwKCSV6ucbKfL
To: /content/Brand_data.zip
100% 36.1M/36.1M [00:02<00:00, 13.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=16wXoUU8MTQbgnYVgsa14vy00nTxwpcKs
To: /content/Brand_Data2.zip
100% 21.1M/21.1M [00:01<00:00, 12.8MB/s]


In [2]:
import os
import zipfile
import shutil
import yaml
from bs4 import BeautifulSoup

# Kaggle API

In [3]:
# # Look at the tutorial on youtube
# from google.colab import files
# files.upload()

# print("Please upload kaggle.json file")
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600

# Download dataset

In [4]:
zip_paths = ['LogoDet-64.zip', 'Brand_data.zip', 'Brand_Data2.zip']

for zip_path in zip_paths:
  print("Unzipping the dataset...")
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall()
  print("Unzipped the dataset successfully.")

Unzipping the dataset...
Unzipped the dataset successfully.
Unzipping the dataset...
Unzipped the dataset successfully.
Unzipping the dataset...
Unzipped the dataset successfully.


# Prepare data

### Select the brands for training

In [5]:
Transportations = ['BMW', 'BUGATTI', 'mclaren', 'Maserati', 'Hyundai-1', 'Lamborghini', 'Bentley', 'Chevrolet', 'Isuzu-1', 'land rover', 'lexus-1', 'Mazda', 'saab', 'toyota', 'vw']
Electronics = ['AKG', 'ASUS', 'Apple', 'BlackBerry', 'BenQ', 'BELKIN-1', 'Canon', 'EPSON', 'Ericsson', 'game boy', 'haier', 'HARMAN KARDON', 'Hisense', 'HTC', 'IBM', 'Logitech',
               'MediaTek', 'meizu', 'roland', 'skyworth', 'viewsonic', 'xiaomi']
Clothes = ['Balenciaga', 'Armani', 'Birkenstock', 'Bottega Veneta', 'Bulgari', 'Calvin Klein', 'chanel', 'Celine', 'Champion', 'Chloe', 'Columbia', 'Converse', 'Guess', 'Guy Laroche',
           'Hugo Boss', 'Kenzo', 'levis', 'miu miu', 'new balance-1', 'patek philippe', 'playboy', 'prada', 'tom ford', 'tommy hilfiger', 'uniqlo', 'valentino\'s', 'versace']

selected_brands = Transportations + Electronics + Clothes

selected_brand_path = [os.path.join('/content/LogoDet-64', x) for x in selected_brands]

min_brand_image = 1e9
for brand_path in selected_brand_path:
  num_of_img = len(os.listdir(brand_path))

  if(num_of_img < min_brand_image):
    min_brand_image = num_of_img

print(min_brand_image//2)

10


### Prepare .ymal file

In [6]:
data = {
  'path': '/content/LogoDet_prep',  # dataset root dir
  'train': 'images/train',  # train images (relative to 'path') 128 images
  'val': 'images/val',  # val images (relative to 'path') 128 images
  'test': 'images/test',  # test images (optional)

  # Classes
  'names': {}
}

for i in range(len(selected_brands)):
  data['names'].update({i:selected_brands[i].replace('-1', '')})

yaml_output = yaml.dump(data, sort_keys=False)

def write_yaml_to_file(py_obj,filename):
    with open(f'{filename}.yaml', 'w',) as f :
        yaml.dump(py_obj,f,sort_keys=False)
    print('Written to file successfully')

print(yaml_output)
write_yaml_to_file(data, 'LogoDept_prep')

path: /content/LogoDet_prep
train: images/train
val: images/val
test: images/test
names:
  0: BMW
  1: BUGATTI
  2: mclaren
  3: Maserati
  4: Hyundai
  5: Lamborghini
  6: Bentley
  7: Chevrolet
  8: Isuzu
  9: land rover
  10: lexus
  11: Mazda
  12: saab
  13: toyota
  14: vw
  15: AKG
  16: ASUS
  17: Apple
  18: BlackBerry
  19: BenQ
  20: BELKIN
  21: Canon
  22: EPSON
  23: Ericsson
  24: game boy
  25: haier
  26: HARMAN KARDON
  27: Hisense
  28: HTC
  29: IBM
  30: Logitech
  31: MediaTek
  32: meizu
  33: roland
  34: skyworth
  35: viewsonic
  36: xiaomi
  37: Balenciaga
  38: Armani
  39: Birkenstock
  40: Bottega Veneta
  41: Bulgari
  42: Calvin Klein
  43: chanel
  44: Celine
  45: Champion
  46: Chloe
  47: Columbia
  48: Converse
  49: Guess
  50: Guy Laroche
  51: Hugo Boss
  52: Kenzo
  53: levis
  54: miu miu
  55: new balance
  56: patek philippe
  57: playboy
  58: prada
  59: tom ford
  60: tommy hilfiger
  61: uniqlo
  62: valentino's
  63: versace

Written to 

### Delete "LogoDept_prep" and "runs" folder

In [ ]:
# # Replace '/path/to/your/folder' with the path to the folder you want to delete
# folder_to_delete1 = '/content/LogoDet_prep'
# folder_to_delete2 = '/content/runs'

# shutil.rmtree(folder_to_delete1)
# shutil.rmtree(folder_to_delete2)

### Create directory for dataset

In [7]:
# Specify the path of the new folder
data_prep = "/content/LogoDet_prep"
images_folder = "/content/LogoDet_prep/images"
labels_folder = "/content/LogoDet_prep/labels"

images_train_folder = "/content/LogoDet_prep/images/train"
images_val_folder = "/content/LogoDet_prep/images/val"
images_test_folder = "/content/LogoDet_prep/images/test"

labels_train_folder = "/content/LogoDet_prep/labels/train"
labels_val_folder = "/content/LogoDet_prep/labels/val"
labels_test_folder = "/content/LogoDet_prep/labels/test"

# Create the new folder
os.makedirs(data_prep, exist_ok=True)
os.makedirs(images_folder, exist_ok=True)
os.makedirs(labels_folder, exist_ok=True)

os.makedirs(images_train_folder, exist_ok=True)
os.makedirs(images_val_folder, exist_ok=True)
os.makedirs(images_test_folder, exist_ok=True)

os.makedirs(labels_train_folder, exist_ok=True)
os.makedirs(labels_val_folder, exist_ok=True)
os.makedirs(labels_test_folder, exist_ok=True)

### Format the data to the dataset folder for YOLOv8 training

In [8]:
def xml2txt(class_no, xml_path, lables_folder_path):
  with open(xml_path, 'r') as f:
      data = f.read()

  Bs_data = BeautifulSoup(data, "xml")
  width = int(Bs_data.find('size').find('width').text)
  height = int(Bs_data.find('size').find('height').text)

  obj_tag_list = Bs_data.find_all('object')

  for obj_tag in obj_tag_list:
    bnd_tag = obj_tag.find('bndbox')
    xmin = int(bnd_tag.find('xmin').text)
    ymin = int(bnd_tag.find('ymin').text)
    xmax = int(bnd_tag.find('xmax').text)
    ymax = int(bnd_tag.find('ymax').text)

    xcenter_ratio = round(((xmin+xmax)/2)/width, 6)
    ycenter_ratio = round(((ymin+ymax)/2)/height, 6)
    width_ratio = round((xmax-xmin)/width, 6)
    height_ratio = round((ymax-ymin)/height, 6)
    extract_data = str(class_no) + ' ' + str(xcenter_ratio) + ' ' + str(ycenter_ratio) + ' ' + str(width_ratio) + ' ' + str(height_ratio) + '\n'

    text_file = open(lables_folder_path, "a")
    text_file.write(extract_data)
    text_file.close()

# class_no = 999
# xml_path = '/content/LogoDet-3K/Clothes/new balance-1/142.xml'
# lables_folder_path = '/content/new balance-1-142.txt'
# xml2txt(class_no, xml_path, lables_folder_path)

In [9]:
from sklearn.model_selection import train_test_split

# Detemine ratio of train, val, test
train_ratio = 1
val_ratio = 0

# Iterate through the input folder
for class_no in range(len(selected_brand_path)):
    brand_name_path = selected_brand_path[class_no]

    # Iterate through images and corresponding XML files
    image_dir_list = [x for x in os.listdir(brand_name_path) if x.endswith('.jpg')]

    # file_count = len(image_dir_list)
    # num_train = round(file_count*train_ratio)
    # num_val = round(file_count*val_ratio)

    # image_dir_train, image_dir_val = train_test_split(image_dir_list, test_size=(val_ratio))

    for image_file in image_dir_list:
        image_path = os.path.join(brand_name_path, image_file)
        xml_file = image_file.replace('.jpg', '.xml')
        text_file = xml_file.replace('.xml', '.txt')
        xml_path = os.path.join(brand_name_path, xml_file)

        images_folder_path = os.path.join("/content/LogoDet_prep/images/train", selected_brands[class_no] + '-' + image_file)
        lables_folder_path = os.path.join("/content/LogoDet_prep/labels/train", selected_brands[class_no] + '-' + text_file)

        xml2txt(class_no, xml_path, lables_folder_path)

        # Copy the image and XML file to the new folder
        shutil.copy(image_path, images_folder_path)

    # for image_file in image_dir_val:
    #     image_path = os.path.join(brand_name_path, image_file)
    #     xml_file = image_file.replace('.jpg', '.xml')
    #     text_file = xml_file.replace('.xml', '.txt')
    #     xml_path = os.path.join(brand_name_path, xml_file)

    #     images_folder_path = os.path.join("/content/LogoDet_prep/images/val", selected_brands[class_no] + '-' + image_file)
    #     lables_folder_path = os.path.join("/content/LogoDet_prep/labels/val", selected_brands[class_no] + '-' + text_file)

    #     xml2txt(class_no, xml_path, lables_folder_path)

    #     # Copy the image and XML file to the new folder
    #     shutil.copy(image_path, images_folder_path)

    # for image_file in image_dir_test:
    #     image_path = os.path.join(brand_name_path, image_file)
    #     xml_file = image_file.replace('.jpg', '.xml')
    #     text_file = xml_file.replace('.xml', '.txt')
    #     xml_path = os.path.join(brand_name_path, xml_file)

    #     images_folder_path = os.path.join("/content/LogoDet_prep/images/test", brand_name + '-' + image_file)
    #     lables_folder_path = os.path.join("/content/LogoDet_prep/labels/test", brand_name + '-' + text_file)

    #     xml2txt(class_no, xml_path, lables_folder_path)

    #     # Copy the image and XML file to the new folder
    #     shutil.copy(image_path, images_folder_path)

print(class_no)

63


In [10]:
def checkIfImage(image):
  return image.endswith('.jpg') or image.endswith('.png') or image.endswith('.jpeg') or image.endswith('.webp')

def checkIfLabel(label):
  return label.endswith('.txt')

for TestData_folder in ['/content/Brand_data/Brand_data', '/content/Brand_Data2']:
  images_folder = os.path.join(TestData_folder, 'images')
  labels_folder = os.path.join(TestData_folder, 'labels')

  for image in os.listdir(images_folder):
    if(checkIfImage(image)):
      image_path = os.path.join(images_folder, image)
      shutil.copy(image_path, '/content/LogoDet_prep/images/val')

  for label in os.listdir(labels_folder):
    if(checkIfLabel(label)):
      label_path = os.path.join(labels_folder, label)
      shutil.copy(label_path, '/content/LogoDet_prep/labels/val')

print(len(os.listdir('/content/LogoDet_prep/images/val')))
print(len(os.listdir('/content/LogoDet_prep/labels/val')))

192
192


In [11]:
# Specify the path of the folder you want to zip
folder_path = '/content/LogoDet_prep'

# Specify the path and name of the zip file you want to create
zip_file_path = '/content/LogoDet_prep.zip'

# Create a zip file
shutil.make_archive(zip_file_path, 'zip', folder_path)

# Move the zip file to the desired location (optional)
shutil.move(zip_file_path + '.zip', zip_file_path)

'/content/LogoDet_prep.zip'

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

zip_file_path = '/content/LogoDet_prep.zip'
save_path = F"/content/gdrive/My Drive/LogoDet_prep.zip"
shutil.move(zip_file_path, save_path)

Mounted at /content/gdrive


'/content/gdrive/My Drive/LogoDet_prep.zip'

In [ ]:
print('training images =',len(os.listdir('/content/LogoDet_prep/images/train')))
print('validating images =',len(os.listdir('/content/LogoDet_prep/images/val')))

training images = 2986
validating images = 192


# Train the model

In [ ]:
!gdown 1-BscyjKMVbBUPDtxzLBBUH0iLqd7wlwk

Downloading...
From: https://drive.google.com/uc?id=1-BscyjKMVbBUPDtxzLBBUH0iLqd7wlwk
To: /content/train5_11-12-2023.zip
100% 87.0M/87.0M [00:01<00:00, 76.6MB/s]


In [ ]:
zip_path = 'train5_11-12-2023.zip'
os.makedirs('/content/runs/detect/train5')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/runs/detect/train5')

In [ ]:
from ultralytics import YOLO

# Adjust the parameters
epochs = 20
imgsz = 960
lr0 = 0.000002
lrf = 0.0001
optimizer = 'Adam'

In [ ]:
# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
# model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolov8s.pt")
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Use the model
model.train(data="LogoDept_prep.yaml", epochs=epochs, imgsz=imgsz, lr0=lr0, lrf=lrf, optimizer=optimizer)  # train the model
path = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train/weights/best.pt, data=LogoDept_prep.yaml, epochs=20, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

train: Scanning /content/LogoDet_prep/labels/train.cache... 2986 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2986/2986 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/LogoDet_prep/labels/val.cache... 192 images, 0 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: Adam(lr=2e-06, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 960 train, 960 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      9.88G     0.6004     0.3598      1.075         12        960: 100%|██████████| 187/187 [02:03<00:00,  1.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:05<00:00,  1.14it/s]

                   all        192        192      0.593        0.5      0.519      0.265



20 epochs completed in 0.792 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 22.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11169680 parameters, 0 gradients, 29.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:10<00:00,  1.74s/it]


                   all        192        192       0.63      0.513      0.546      0.273
                   BMW        192          3       0.64      0.667      0.567      0.296
               BUGATTI        192          3      0.754      0.333      0.567      0.397
               mclaren        192          3      0.968          1      0.995      0.308
              Maserati        192          3      0.233      0.333       0.19       0.14
               Hyundai        192          3          1          0          0          0
           Lamborghini        192          3      0.888          1      0.995      0.814
               Bentley        192          3          1       0.49      0.671      0.472
             Chevrolet        192          3      0.576          1      0.731      0.407
                 Isuzu        192          3          1          0       0.39     0.0781
            land rover        192          3       0.78      0.333      0.382      0.306
                 lexu

# Evaluate and Test

In [ ]:
model = YOLO('/content/runs/detect/train5/weights/best.pt')
metrics = model.val(imgsz=960)  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11169680 parameters, 0 gradients, 29.7 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 23.7MB/s]
val: Scanning /content/LogoDet_prep/labels/val... 192 images, 0 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<00:00, 1105.64it/s]


val: New cache created: /content/LogoDet_prep/labels/val.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [05:22<00:00, 26.84s/it]


                   all        192        192      0.641      0.511      0.554      0.265
                   BMW        192          3      0.789      0.333      0.564      0.295
               BUGATTI        192          3      0.782      0.333      0.376      0.264
               mclaren        192          3      0.963          1      0.995      0.311
              Maserati        192          3      0.083      0.111      0.121     0.0664
               Hyundai        192          3          1          0     0.0828    0.00828
           Lamborghini        192          3      0.904          1      0.995      0.796
               Bentley        192          3          1      0.627      0.671      0.439
             Chevrolet        192          3      0.495          1      0.597      0.323
                 Isuzu        192          3          1          0      0.348     0.0348
            land rover        192          3          1      0.463      0.713      0.467
                 lexu

array([    0.29458,     0.26352,     0.31134,     0.06636,   0.0082793,       0.796,     0.43859,     0.32275,    0.034776,     0.46728,     0.22648,     0.46062,     0.51061,     0.30509,     0.17623,     0.10446,     0.36283,    0.056571,    0.078853,     0.38414,      0.1703,     0.15478,      0.4811,           0,
           0.34636,     0.39005,    0.089251,    0.076727,     0.27432,     0.36901,     0.69765,           0,       0.322,     0.86474,     0.02093,     0.18416,     0.22314,      0.0995,           0,     0.10054,     0.59982,     0.19119,     0.45158,     0.11676,     0.30078,     0.48619,     0.46584,    0.026038,
                 0,      0.4229,     0.59841,     0.43783,     0.16897,     0.61043,     0.41208,           0,           0,           0,     0.10169,    0.022702,     0.29201,     0.14137,           0,     0.60407])

In [ ]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
# model = YOLO('/content/yolov8n.pt')

# Define path to the image file
test_path = '/content/LogoDet_prep/images/val'
file_list = os.listdir(test_path)
source = [os.path.join(test_path, file_name) for file_name in file_list]

# Run inference on the source
results = model(source, imgsz=640, save=True)  # list of Results objects

NameError: ignored

# Save model to drive

In [ ]:
import torch
from google.colab import drive
drive.mount('/content/gdrive')

model = YOLO('/content/runs/detect/train/weights/best.pt')
model_save_name = 'Brand-Detection64classes_11-12-2023-2214.pt' # Change every time when you want to save a new model to drive
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

Mounted at /content/gdrive


In [ ]:
# Specify the path of the folder you want to zip
folder_path = '/content/runs/detect/train5'

# Specify the path and name of the zip file you want to create
zip_file_path = '/content/train5_11-12-2023.zip'

# Create a zip file
shutil.make_archive(zip_file_path, 'zip', folder_path)

# Move the zip file to the desired location (optional)
shutil.move(zip_file_path + '.zip', zip_file_path)

'/content/train5_11-12-2023.zip'

In [ ]:
zip_file_path = '/content/train5_11-12-2023.zip'
save_path = F"/content/gdrive/My Drive/train5_11-12-2023.zip"
shutil.move(zip_file_path, save_path)

'/content/gdrive/My Drive/train5_11-12-2023.zip'